In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression,Ridge, RidgeCV, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error

In [2]:
!pip install --upgrade scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sklearn
print(sklearn.__version__)

1.3.1


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('final_data_for_modelling.csv')

In [6]:
df.head()

,brand_name,model,rating,has_dual_sim,has_5G,has_NFC,has_IR_Blaster,processor_brand,no_of_cores,processor_speed,ram,rom,fast_charging_available,battery_capacity,screen_size,resolution,refresh_rate,no_of_rear_cameras,no_of_front_cameras,primary_rear_camera,primary_front_camera,memory_extended_upto,operating_system,price
0,xiaomi,Xiaomi Redmi 12 5G,75,1,1,0,1,snapdragon,8,2.2,4,128,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android,11999
1,xiaomi,Xiaomi Redmi 12 5G (6 GB RAM + 128 GB),77,1,1,0,1,snapdragon,8,2.2,6,128,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android,13499
2,xiaomi,Xiaomi Redmi 12 5G (8GB RAM + 256GB),80,1,1,0,1,snapdragon,8,2.2,8,256,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android,15499
3,oneplus,OnePlus Nord CE 3 Lite 5G,85,1,1,0,0,snapdragon,8,2.2,8,128,1,5000,6.72,Ultra HD,120,3,1,108,16,1024,Android,19999
4,samsung,Samsung Galaxy F34 5G,83,1,1,1,0,exynos,8,2.4,6,128,1,6000,6.50,Ultra HD,120,3,1,50,13,1024,Android,18999


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   brand_name               696 non-null    object 
 1   model                    696 non-null    object 
 2   rating                   696 non-null    int64  
 3   has_dual_sim             696 non-null    int64  
 4   has_5G                   696 non-null    int64  
 5   has_NFC                  696 non-null    int64  
 6   has_IR_Blaster           696 non-null    int64  
 7   processor_brand          696 non-null    object 
 8   no_of_cores              696 non-null    int64  
 9   processor_speed          696 non-null    float64
 10  ram                      696 non-null    int64  
 11  rom                      696 non-null    int64  
 12  fast_charging_available  696 non-null    int64  
 13  battery_capacity         696 non-null    int64  
 14  screen_size              6

In [8]:
X = df.drop(columns=['price','model','rating'])
y = df['price']

In [9]:
X

,brand_name,has_dual_sim,has_5G,has_NFC,has_IR_Blaster,processor_brand,no_of_cores,processor_speed,ram,rom,fast_charging_available,battery_capacity,screen_size,resolution,refresh_rate,no_of_rear_cameras,no_of_front_cameras,primary_rear_camera,primary_front_camera,memory_extended_upto,operating_system
0,xiaomi,1,1,0,1,snapdragon,8,2.20,4,128,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android
1,xiaomi,1,1,0,1,snapdragon,8,2.20,6,128,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android
2,xiaomi,1,1,0,1,snapdragon,8,2.20,8,256,1,5000,6.79,Ultra HD,90,2,1,50,8,1024,Android
3,oneplus,1,1,0,0,snapdragon,8,2.20,8,128,1,5000,6.72,Ultra HD,120,3,1,108,16,1024,Android
4,samsung,1,1,1,0,exynos,8,2.40,6,128,1,6000,6.50,Ultra HD,120,3,1,50,13,1024,Android
5,oneplus,1,1,1,1,snapdragon,8,2.70,8,128,1,5000,6.70,Ultra HD,120,3,1,50,16,1024,Android
6,vivo,1,1,0,0,mediatek,8,2.20,6,128,1,5000,6.58,Ultra HD,60,2,1,50,8,1024,Android
7,realme,1,0,0,0,unisoc,8,1.82,4,128,1,5000,6.74,Ultra HD,90,2,1,108,8,2048,Android
8,motorola,1,1,1,0,mediatek,8,2.60,8,256,1,4400,6.55,Ultra HD,144,2,1,50,32,0,Android
9,oppo,1,1,1,1,mediatek,8,2.60,8,256,1,5000,6.70,Ultra HD,120,3,1,64,32,128,Android


In [10]:
y

0       11999
1       13499
2       15499
3       19999
4       18999
5       26999
6       13999
7        9999
8       29999
9       32999
10      23599
11      39999
12       9999
13      32674
14      27599
15      21999
16      18999
17      17999
18      38684
19      32799
20      12999
21      16999
22      14999
23      21993
24       9999
25      26683
26      29999
27      10999
28     106500
29      13990
30      16999
31      28999
32      39999
33      17699
34      13699
35      61999
36      16999
37      19989
38      28999
39      54799
40      15999
41      20999
42      11999
43      68999
44      34999
45      15999
46      29599
47       8499
48      18999
49     127999
50       8999
51      20999
52       6999
53      13499
54      20999
55      14999
56      37999
57      24999
58      20999
59      38999
60      15349
61      26999
62      79999
63      17499
64      21999
65      15998
66      17960
67      27489
68      84999
69      14277
70      18999
71    

In [11]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [12]:
#OHE

In [13]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['has_dual_sim', 'has_5G', 'has_NFC', 'has_IR_Blaster', 'no_of_cores', 'processor_speed', 'ram', 'rom', 'fast_charging_available', 'battery_capacity', 'screen_size', 'refresh_rate', 'no_of_rear_cameras', 'no_of_front_cameras', 'primary_front_camera', 'primary_rear_camera', 'memory_extended_upto']),
        ('cat', OneHotEncoder(drop='first'),['brand_name','processor_brand','operating_system']),
        ('cat1', OrdinalEncoder(categories=[['FWVGA','HD','Full HD','Full HD+','Quad HD','Ultra HD']]),['resolution'])
    ], 
    remainder='passthrough'
)

In [14]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num', StandardScaler(),
                                 ['has_dual_sim', 'has_5G', 'has_NFC',
                                  'has_IR_Blaster', 'no_of_cores',
                                  'processor_speed', 'ram', 'rom',
                                  'fast_charging_available', 'battery_capacity',
                                  'screen_size', 'refresh_rate',
                                  'no_of_rear_cameras', 'no_of_front_cameras',
                                  'primary_front_camera', 'primary_rear_camera',
                                  'memory_extended_upto']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['brand_name', 'processor_brand',
                                  'operating_system']),
                                ('cat1',
                                 OrdinalEncoder(categories=[['FWVGA', 'HD',
                                                             'Full HD',
                                                             'Full HD+',
                                                             'Quad HD',
                                                             'Ultra HD']]),
                                 ['resolution'])])

In [15]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('random forest',RandomForestRegressor())
])

In [16]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [17]:
scores.mean()

0.9589505114113608

In [18]:
scores.std()

0.010968028763527678

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [20]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['has_dual_sim', 'has_5G',
                                                   'has_NFC', 'has_IR_Blaster',
                                                   'no_of_cores',
                                                   'processor_speed', 'ram',
                                                   'rom',
                                                   'fast_charging_available',
                                                   'battery_capacity',
                                                   'screen_size',
                                                   'refresh_rate',
                                                   'no_of_rear_cameras',
                                                   'no_of_front_cameras',
                                                   'primary_front_camera',
                                                   'primary_rear_camera',
                                                   'memory_extended_upto']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['brand_name',
                                                   'processor_brand',
                                                   'operating_system']),
                                                 ('cat1',
                                                  OrdinalEncoder(categories=[['FWVGA',
                                                                              'HD',
                                                                              'Full '
                                                                              'HD',
                                                                              'Full '
                                                                              'HD+',
                                                                              'Quad '
                                                                              'HD',
                                                                              'Ultra '
                                                                              'HD']]),
                                                  ['resolution'])])),
                ('random forest', RandomForestRegressor())])

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
y_pred = np.expm1(y_pred)

In [23]:
mean_absolute_error(np.expm1(y_test),y_pred)

3787.5999963934332

In [24]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [25]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'xgboost':XGBRegressor()
}

In [26]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [27]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [28]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,9.617549e-01,2723.142235
9,xgboost,9.687731e-01,3074.127637
1,svr,9.086831e-01,3710.108391
5,random forest,9.579525e-01,3805.611472
7,gradient boosting,9.603061e-01,3819.478915
0,linear_reg,-9.706882e+22,4421.634927
2,ridge,9.322034e-01,4484.404446
4,decision tree,9.178311e-01,4650.715302
8,adaboost,9.274055e-01,4852.585873
3,LASSO,-1.740863e-02,12806.540148


## Exporting the model

In [29]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['has_dual_sim', 'has_5G', 'has_NFC', 'has_IR_Blaster', 'no_of_cores', 'processor_speed', 'ram', 'rom', 'fast_charging_available', 'battery_capacity', 'screen_size', 'refresh_rate', 'no_of_rear_cameras', 'no_of_front_cameras', 'primary_front_camera', 'primary_rear_camera', 'memory_extended_upto']),
        ('cat', OneHotEncoder(drop='first'),['brand_name','processor_brand','operating_system']),
        ('cat1', OrdinalEncoder(categories=[['FWVGA','HD','Full HD','Full HD+','Quad HD','Ultra HD']]),['resolution'])
    ], 
    remainder='passthrough'
)

In [30]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ExtraTreesRegressor())
])

In [31]:
pipeline.fit(X,y_transformed)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['has_dual_sim', 'has_5G',
                                                   'has_NFC', 'has_IR_Blaster',
                                                   'no_of_cores',
                                                   'processor_speed', 'ram',
                                                   'rom',
                                                   'fast_charging_available',
                                                   'battery_capacity',
                                                   'screen_size',
                                                   'refresh_rate',
                                                   'no_of_rear_cameras',
                                                   'no_of_front_cameras',
                                                   'primary_front_camera',
                                                   'primary_rear_camera',
                                                   'memory_extended_upto']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['brand_name',
                                                   'processor_brand',
                                                   'operating_system']),
                                                 ('cat1',
                                                  OrdinalEncoder(categories=[['FWVGA',
                                                                              'HD',
                                                                              'Full '
                                                                              'HD',
                                                                              'Full '
                                                                              'HD+',
                                                                              'Quad '
                                                                              'HD',
                                                                              'Ultra '
                                                                              'HD']]),
                                                  ['resolution'])])),
                ('regressor', ExtraTreesRegressor())])

In [32]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [33]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [34]:
# Trying out Prediction

In [35]:
data = [['xiaomi', 1, 0, 0, 0,'mediatek', 8, 2.0, 4, 64, 0, 5000, 6.71, 'Quad HD', 60, 2, 1, 5, 50, 1024, 'Android']]
columns = ['brand_name', 'has_dual_sim', 'has_5G', 'has_NFC','has_IR_Blaster','processor_brand','no_of_cores','processor_speed',
           'ram','rom','fast_charging_available','battery_capacity','screen_size','resolution','refresh_rate',
           'no_of_rear_cameras','no_of_front_cameras','primary_front_camera','primary_rear_camera','memory_extended_upto',
           'operating_system']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,brand_name,has_dual_sim,has_5G,has_NFC,has_IR_Blaster,processor_brand,no_of_cores,processor_speed,ram,rom,fast_charging_available,battery_capacity,screen_size,resolution,refresh_rate,no_of_rear_cameras,no_of_front_cameras,primary_front_camera,primary_rear_camera,memory_extended_upto,operating_system
0,xiaomi,1,0,0,0,mediatek,8,2.0,4,64,0,5000,6.71,Quad HD,60,2,1,5,50,1024,Android


In [36]:
np.expm1(pipeline.predict(one_df))

array([8499.])

In [37]:
X.dtypes

brand_name                  object
has_dual_sim                 int64
has_5G                       int64
has_NFC                      int64
has_IR_Blaster               int64
processor_brand             object
no_of_cores                  int64
processor_speed            float64
ram                          int64
rom                          int64
fast_charging_available      int64
battery_capacity             int64
screen_size                float64
resolution                  object
refresh_rate                 int64
no_of_rear_cameras           int64
no_of_front_cameras          int64
primary_rear_camera          int64
primary_front_camera         int64
memory_extended_upto         int64
operating_system            object
dtype: object

In [38]:
sorted(df['brand_name'].unique().tolist())

['apple',
 'google',
 'ikall',
 'infinix',
 'iqoo',
 'itel',
 'lava',
 'motorola',
 'nokia',
 'nothing',
 'oneplus',
 'oppo',
 'poco',
 'realme',
 'samsung',
 'tecno',
 'vivo',
 'xiaomi']

In [421]:
X.dtypes

brand_name                  object
has_dual_sim                 int64
has_5G                       int64
has_NFC                      int64
has_IR_Blaster               int64
processor_brand             object
no_of_cores                  int64
processor_speed            float64
ram                          int64
rom                          int64
fast_charging_available      int64
battery_capacity             int64
screen_size                float64
resolution                  object
refresh_rate                 int64
no_of_rear_cameras           int64
no_of_front_cameras          int64
primary_front_camera         int64
primary_rear_camera          int64
memory_extended_upto         int64
operating_system            object
dtype: object

In [422]:
sorted(df['ram'].unique().tolist(),reverse=True)

[16, 12, 8, 6, 4, 3, 2, 1]

In [296]:
sorted(df['memory_extended_upto'].unique().tolist(),reverse=True)

[2048, 1024, 512, 256, 128, 64, 32, 0]